In [1]:
import os
os.chdir('c:\\Users\\sedar\\Documents\\UCL\\1_BIOL0041_MSci_project\\')
from actin_meshwork_analysis.meshwork.utils import list_files_dir_str
import numpy as np 
import matplotlib.pyplot as plt
from actin_meshwork_analysis.meshwork.actinimg import get_ActinImg

from ipywidgets import interactive, FloatSlider, IntSlider, HBox, interactive_output

In [ ]:
# list and load files
data_path = os.path.join(os.getcwd(), "actin_meshwork_analysis/process_data/sample_data")
os.path.exists(data_path)

all_filenames, all_filepaths = list_files_dir_str(data_path)
celltype = 'Untransduced'
filename, filepath = all_filenames[celltype][0], all_filepaths[celltype][0] 

# sample image
actimg = get_ActinImg(filename, os.path.join(data_path,celltype))

In [ ]:
def plot_threshold(threshold):
    theta_x6 = np.arange(0,360,60)

    actimg = get_ActinImg(filename, os.path.join(data_path,celltype))
    actimg.normalise()
    actimg.steerable_gauss_2order_thetas(thetas=theta_x6,sigma=2,substack=[1,3],visualise=False)
    actimg.z_project_min()
    actimg.threshold(threshold)

    actimg.visualise('manipulated')

threshold = 0.0001

threshold_widget = FloatSlider(min=0.0001, max=0.009, step=0.0005, value=threshold, readout_format='.5f')
threshold_widget.continuous_update=False

interactive(plot_threshold, threshold=threshold_widget)

In [2]:
from skimage.measure import profile_line
from matplotlib_scalebar.scalebar import ScaleBar
data_path = os.path.join(os.getcwd(), "actin_meshwork_analysis/process_data/sample_data/CARs")
filename='3min_FOV3_decon.tif'

In [3]:
def plot_threshold(threshold, theta):

    actimg = get_ActinImg(filename, data_path) # base = [1,4], cyto = [4,7] 
    actimg.normalise()
    actimg.steerable_gauss_2order(theta=theta,sigma=2,substack=[1,4],visualise=False)
    linprof = profile_line(actimg.manipulated_stack, (0,0), actimg.shape)
    linprof = linprof[np.argwhere(linprof>0)]
    thresh = np.percentile(linprof, threshold)
    actimg.z_project_min()
    actimg.threshold(threshold=thresh)
    
    plt.figure()
    plt.subplot(1,1,1)
    plt.imshow(actimg.manipulated_stack,cmap='gray')
    scalebar = ScaleBar(actimg.resolution, 'nm', box_color='None', color='#F2F2F2', location='lower left') 
    plt.gca().add_artist(scalebar)
    plt.axis('off')
    plt.plot((0,actimg.shape[0]),(0,actimg.shape[1]), color='white')
    # plt.subplot(1,2,2)
    # plt.plot(interm) # linprof
    # plt.ylim(0,1.1*np.max(linprof))
    # plt.xlabel('pixel location on line')
    # plt.ylabel('intensity')
    # plt.title('line profile after steerable Gaussian filter')
    # plt.tight_layout()
    

threshold = 0.0001
theta = 0

threshold_widget = IntSlider(min=90, max=99, step=1, value=threshold, continuous_update=False)
theta_widget = FloatSlider(min=0, max=180, step=15, value=theta, readout_format='.0f', continuous_update=False)

interactive(plot_threshold, threshold=threshold_widget, theta=theta_widget)

interactive(children=(IntSlider(value=90, continuous_update=False, description='threshold', max=99, min=90), F…